# TensorFlow Tutorial - 7. Reinforcemet Learning

본 문서는 TensorFlow 를 사용하여 Deep Learning을 구현하기 위한 기초적인 실습 자료이다. 첫 번째 파트에서는 tensorflow에 대한 기본적인 설명과 deep learning 예제를 다루어보고, 두 번째 파트에서는 오픈소스를 활용한 Deep Reinforcement Learning 을 실습해보는 시간을 갖는다. 마지막으로는 TensorFlow로 구현되고 공개된 여러 오픈소스를 둘러본다.

The code and comments are written by Dong-Hyun Kwak <imcomking@gmail.com>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.





# Agenda
## [1. Tensorflow Intro](../TensorflowIntro/TensorflowIntro.ipynb)
## [2. Basic](../Basic/Basic.ipynb)
## [3. Gradient Descent](../GradientDescent/GradientDescent.ipynb)
## [4. MLP](../MLP/MLP.ipynb)
## [5. RNN](../RNN/RNN.ipynb)
## [6. CNN](../CNN/CNN.ipynb)
## [7. Reinforcemet Learning](../ReinforcemetLearning/ReinforcemetLearning.ipynb)



## Part 2. Deep Reinforcement Learing

Deep Reinforcement Learning이란, 기존의 강화학습에서 Q function을 딥러닝으로 근사하는 모델을 의미한다. 대표적으로 구글 Deep Mind의 Atari와 최근에 화제가 된 AlphaGo 역시 이 Deep Reinforcement Learning 응용의 한가지이다.

이번 파트에서는 Deep Reinforcement Learning을 이용해서 간단한 2차원 게임을 플레이하고, reward로 부터 스스로 학습하는 Kaparthy의 오픈소스 예제를 실습해 본다.

<img alt="Deep RL" style="border-width:0" width="600" src="example.gif?raw=true" />


(출처: https://github.com/nivwusquorum/tensorflow-deepq)

### Reinforcement Learning

Reinforcement Learning, 이하 RL은 supervised learning과 달리 데이터에 대한 정확한 정답을 받지 않고, 내가 한 행동에 대한 reward feedback 만으로 학습을 수행하는 알고리즘이다. 이를 강화학습이라 부르며, 이것을 수행하는 가장 대표적인 알고리즘으로 Q-Learning 이 있다.

RL을 이해하는 것은 매우 많은 공부를 필요로 하기 때문에, 우선 2D게임을 예로 들어 아주 기본적인 개념만 살펴본다.

* **State**: 게임에서의 각 물체들의 위치, 속도, 벽과의 거리 등을 의미한다.
* **Action**: 게임을 플레이하는 주인공의 행동을 의미한다. 여기서는 4가지 방향에 대한 움직임이 이에 해당한다.
* **Reward**: 게임을 플레이하면서 받는 score. 여기서는 초록색을 먹으면 +1 , 빨간색을 먹으면 -1을 점수로 받는다.
* **Value**: 해당 action 또는 state가 미래에 얼마나 큰 reward를 가져올 지에 대한 예측값.
* **Policy**: 주인공이 현재의 게임 State에서 Reward를 최대한 얻기 위해 Action을 선택하는 전략. 한마디로 [게임을 플레이하는 방법]자체를 의미한다.




<img src='https://omarsbrain.files.wordpress.com/2010/02/rl.png'>
(출처 : Sutton, 1998, Reinforcement Learning: An Introduction)

<img src='http://www.randomant.net/wp-content/uploads/2016/05/q_learning3.jpg'>
(출처 : http://www.randomant.net/wp-content/uploads/2016/05/q_learning3.jpg)

<br><br>
아래의 예제 코드를 실행시키기 위해서는 리눅스 shell에서 다음의 명령어를 실행해 필요한 python package를 설치해야한다.
```
pip install future euclid redis matplotlib
```

In [1]:
%matplotlib inline
import tensorflow as tf
tf.reset_default_graph()

from tf_rl.controller import DiscreteDeepQ, HumanController
from tf_rl.simulation import KarpathyGame
from tf_rl import simulate
from tf_rl.models import MLP

from __future__ import print_function

#### Environment Settings

이제 우리가 원하는 게임 환경을 설정하고, 적절한 reward와 object의 개수 및 observation 을 조절한다

In [2]:
current_settings = {
    'objects': [
        'friend',
        'enemy',
    ],
    'colors': {
        'hero':   'yellow',
        'friend': 'green',
        'enemy':  'red',
    },
    'object_reward': {
        'friend': 1,
        'enemy': -1,
    },
    "num_objects": {
        'friend' : 25,
        'enemy' :  25,
    },
    
    'hero_bounces_off_walls': False,
    'world_size': (700,500),
    'hero_initial_position': [400, 300],
    'hero_initial_speed':    [0,   0],
    "maximum_speed":         [50, 50],
    "object_radius": 10.0,
    "num_observation_lines" : 32, # the number of antennas
    "observation_line_length": 240., # the length of antennas
    "tolerable_distance_to_wall": 50, 
    "wall_distance_penalty":  -0.0, # if the hero is close to wall, that receives penalty
    "delta_v": 50 # speed value
}

# create the game simulator
g = KarpathyGame(current_settings)


#### Deep Learning Architecture

이제 Q function을 근사하기 위한 딥러닝 모델을 만들어보자. 이번 예제에서는 위에서 보았던 4층짜리 MLP를 사용한다.

In [3]:
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))

    

journalist = tf.train.SummaryWriter("/tmp/drl")

# Brain maps from observation to Q values for different actions.
# Here it is a done using a multi layer perceptron with 2 hidden layers
brain = MLP([g.observation_size,], [200, 200, g.num_actions],  [tf.tanh, tf.tanh, tf.identity])

#### Make an Agent

이제 Discrete Deep Q learning 알고리즘이 이 게임을 플레이하면서 학습을 하도록 agent로 설정을 한다.

In [ ]:
# The optimizer to use. Here we use RMSProp as recommended by the publication
optimizer = tf.train.RMSPropOptimizer(learning_rate= 0.001, decay=0.9)

# DiscreteDeepQ object
current_controller = DiscreteDeepQ(g.observation_size, g.num_actions, brain, optimizer, session,
                                   discount_rate=0.99, exploration_period=5000, max_experience=10000, 
                                   store_every_nth=4, train_every_nth=4,
                                   summary_writer=journalist)

session.run(tf.initialize_all_variables())
session.run(current_controller.target_network_update)
#journalist.add_graph(session.graph_def)

#### Play the Game

실제로 게임을 플레이하면서 강화학습을 하는 과정을 지켜본다.

In [ ]:
FPS          = 30
ACTION_EVERY = 3
    
fast_mode = True
if fast_mode:
    WAIT, VISUALIZE_EVERY = False, 20
else:
    WAIT, VISUALIZE_EVERY = True, 1

    
try:
    with tf.device("/gpu:0"):
        simulate(simulation=g,
                 controller=current_controller,
                 fps=FPS,
                 visualize_every=VISUALIZE_EVERY,
                 action_every=ACTION_EVERY,
                 wait=WAIT,
                 disable_training=False,
                 simulation_resolution=0.001,
                 save_path=None)
except KeyboardInterrupt:
    print("Interrupted")
    

#session.close()

#### Applications
* Parameter 들을 바꾸어 enemy와 friend의 개수 차이가 최대한 많이 나도록 agent를 학습시켜본다.
* Boss object를 추가해본다.
* Deep RL에서의 tensorboard를 열어서 visualize를 해본다.